In [2]:
import tqdm
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import time

# Access website using Selenium

In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

In [9]:
chrome_path = 'C:/Users/carla/OneDrive/Área de Trabalho/chromedriver'
ser = Service(chrome_path)

In [10]:
driver = webdriver.Chrome(service=ser)

In [11]:
driver.get(url)

It was necessary to access through selenium in order to get the main table with the accidents organized by date. However, it was necessary to get the links from each accident in order to get some important information that was present in each accident page, such as the phase of flight and the classification, among others.

In [91]:
url = 'https://aviation-safety.net/database/dblist.php?Year=2011&lang=&page=1'

In [5]:
from datetime import date
current_year = date.today().year

In [92]:
years_url = []
while int(url[53:57]) <= current_year:
       years_url.append(url)
       url = url[:53]+ str(int(url[53:57])+1) +url[57:]

In [93]:
accidents = pd.DataFrame()
for year in years_url:
    table_all_pages = pd.DataFrame()
    list_url = [year]
    for url in list_url:
        driver.get(url)
        pages = [i.get_attribute('href') for i in driver.find_elements('tag name','a') if 'page'in i.get_attribute('href') ]
        pages = list(set(pages))
        [list_url.append(item) for item in pages]
        list_url = list(set(list_url))
        elem = driver.find_element('tag name','table')
        elem.get_attribute('outerHTML')
        elem.find_element('tag name','a')
        url1 = [i.get_attribute('href') for i in (elem.find_elements('tag name','a'))][1:]
        tab_page1 = pd.read_html(elem.get_attribute('outerHTML'))[0]
        tab_page1['url'] = url1
        table_all_pages = table_all_pages.append(tab_page1)
        time.sleep(5)
    accidents = accidents.append(table_all_pages)

In [95]:
accidents.to_csv('accidents-2011-2017.csv')

In [63]:
years_url = []
while int(url[53:57]) <= current_year:
       years_url.append(url)
       url = url[:53]+ str(int(url[53:57])+1) +url[57:]

In [69]:
accidents.to_csv('accidents-2018-2021.csv', index=False)

In [17]:
driver.get('https://aviation-safety.net/database/record.php?id=20201213-0')

In [36]:
 tabelaA = driver.find_elements('tag name','table')

In [79]:
tabelaA

[<selenium.webdriver.remote.webelement.WebElement (session="f2a8d211c04976ac81687352199bf079", element="c76a8fec-5d12-4052-b319-8672c4d6238e")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f2a8d211c04976ac81687352199bf079", element="d7ffbd4d-3d02-4779-8e86-c35ebf250765")>,
 <selenium.webdriver.remote.webelement.WebElement (session="f2a8d211c04976ac81687352199bf079", element="a453da2e-47c2-4ae2-b0c6-13b8b0630839")>]

In [61]:
conteudo=tabelaA[0].get_attribute('textContent')
conteudo

'\nStatus:Preliminary\nDate:Sunday 13 December 2020\nTime:20:00 \nType:Eclipse 500\nOperator:private\nRegistration: N686TM\nMSN: 000221\nFirst flight: 2008 \nEngines: 2 Pratt & Whitney Canada PW610F-A\nCrew:Fatalities: 0 / Occupants: 1\nPassengers:Fatalities: 0 / Occupants: 0\nTotal:Fatalities: 0 / Occupants: 1 \n\n\n\nAircraft damage: Substantial\nLocation:Leadville Airport, CO (LXV) ( \xa0 United States of America) \n\nPhase: Landing (LDG)\nNature:Private\n\n\nDeparture airport:San Diego-Montgomery Field, CA (MYF/KMYF), United States of America\n\n\nDestination airport:Leadville Airport, CO (LXV/KLXV), United States of America\n\n\n\n\n'

In [69]:
tabelaB = driver.find_elements('tag name','span')
tabelaB[1].get_attribute('textContent')

'The aircraft experienced a runway excursion upon landing at Lake County Airport (LXV/KLXV), Leadville, Colorado.  The airplane sustained substantial damage and the sole pilot onboard was not injured during the incident.'

In [78]:
tabelaC = driver.find_elements('xpath','//*[@id="contentcolumn"]/div/a[1]')
tabelaC[0].get_attribute('textContent')

'Runway excursion'

In [73]:
len(tabelaC)

31

Then, it was necessary to get each HTML from the links in order to create the soup, making it easier to get the information from the dataframe, using the URLs that were already found by the previous function.

In [216]:
def get_html(url):
    try:
        headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36'}
        r = requests.get(url, headers=headers)
        time.sleep(7)
        return r.text
    except:
        return None

In [217]:
acc_complete['html'] = acc_complete['url'].apply(get_html)

In [219]:
acc_complete.to_csv('accidents-2011-2021-html.csv')

In [196]:
teste20 = get_html('https://aviation-safety.net/database/record.php?id=20201213-0')

In [106]:
soup = BeautifulSoup(get_html('https://aviation-safety.net/database/record.php?id=20201213-0'))

In [109]:
soup.find_all('table')[0].text

'\nStatus:Preliminary\nDate:Sunday 13 December 2020\nTime:20:00\nType:Eclipse 500\nOperator:private\nRegistration: N686TM\nMSN: 000221\nFirst flight: 2008\nEngines: 2 Pratt & Whitney Canada PW610F-A\nCrew:Fatalities: 0 / Occupants: 1\nPassengers:Fatalities: 0 / Occupants: 0\nTotal:Fatalities: 0 / Occupants: 1 \nAircraft damage: Substantial\nLocation:Leadville Airport, CO (LXV) ( \xa0 United States of America) \r\n\nPhase: Landing (LDG)\nNature:Private\nDeparture airport:San Diego-Montgomery Field, CA (MYF/KMYF), United States of America\nDestination airport:Leadville Airport, CO (LXV/KLXV), United States of America\n'

In [188]:
text_list_narr =[]
for element in soup.find_all('span'):
    text_list_narr.append(element.text)
print ('\n'.join(text_list_narr))

Narrative:
The aircraft experienced a runway excursion upon landing at Lake County Airport (LXV/KLXV), Leadville, Colorado.  The airplane sustained substantial damage and the sole pilot onboard was not injured during the incident.


In [201]:
text_list =[]
for element in soup.find_all('table'):
    text_list.append(element.text)
    texto = '\n'.join(text_list)
print (texto)


Status:Preliminary
Date:Sunday 13 December 2020
Time:20:00
Type:Eclipse 500
Operator:private
Registration: N686TM
MSN: 000221
First flight: 2008
Engines: 2 Pratt & Whitney Canada PW610F-A
Crew:Fatalities: 0 / Occupants: 1
Passengers:Fatalities: 0 / Occupants: 0
Total:Fatalities: 0 / Occupants: 1 
Aircraft damage: Substantial
Location:Leadville Airport, CO (LXV) (   United States of America) 

Phase: Landing (LDG)
Nature:Private
Departure airport:San Diego-Montgomery Field, CA (MYF/KMYF), United States of America
Destination airport:Leadville Airport, CO (LXV/KLXV), United States of America





Investigating agency: NTSB 
Status: Investigation ongoing
Accident number: CEN21LA086



Investigating agency: NTSB 
Status: Investigation ongoing
Accident number: CEN21LA086



In [20]:
import re

In [123]:
texto2 = '\n'.join(text_list)

In [136]:
teste = [re.sub(':', '', item) for item in re.findall(':.*',texto2)]
teste

['Preliminary',
 'Sunday 13 December 2020',
 '2000',
 'Eclipse 500',
 'private',
 ' N686TM',
 ' 000221',
 ' 2008',
 ' 2 Pratt & Whitney Canada PW610F-A',
 'Fatalities 0 / Occupants 1',
 'Fatalities 0 / Occupants 0',
 'Fatalities 0 / Occupants 1 ',
 ' Substantial',
 'Leadville Airport, CO (LXV) ( \xa0 United States of America) \r',
 ' Landing (LDG)',
 'Private',
 'San Diego-Montgomery Field, CA (MYF/KMYF), United States of America',
 'Leadville Airport, CO (LXV/KLXV), United States of America',
 '\xa0NTSB ',
 ' Investigation ongoing',
 ' CEN21LA086',
 '\xa0NTSB ',
 ' Investigation ongoing',
 ' CEN21LA086']

In [175]:
text_list2 =[]
for element in soup.find_all('div', attrs={'class':'innertube'}):
    for e in element.find_all('a'):
        text_list2.append(e.text)
print ('\n'.join(text_list2))
text_list2[6]

Eclipse 500
private
Pratt & Whitney Canada PW610F-A
United States of America
San Diego-Montgomery Field, CA (MYF/KMYF)
Leadville Airport, CO (LXV/KLXV)
Runway excursion

Tweet


In [195]:
teste = [re.sub(':\xa0', '', item) for item in re.findall(':.*',text_list[2])]
teste[0]

'NTSB '

# First table items

In [44]:
def details(html, index):
    try:
        soup = BeautifulSoup(html)
        text_list =[]
        for element in soup.find_all('table'):
            text_list.append(element.text)
        cleaning = [re.sub(':', '', item) for item in re.findall(':.*',text_list[0])]
        return cleaning[index]
    except:
        return None

In [206]:
details1(teste20, 14)

' Landing (LDG)'

In [41]:
def details_inv(html):
    try:
        soup = BeautifulSoup(html)
        text_list =[]
        for element in soup.find_all('table'):
            text_list.append(element.text)
        cleaning = [re.sub(':\xa0', '', item) for item in re.findall(':.*',text_list[2])]
        return cleaning[0]
    except:
        return None


In [210]:
details2(teste20)

'NTSB '

In [34]:
def details3(html):
    try:
        soup = BeautifulSoup(html)
        text_list_narr =[]
        for element in soup.find_all('span'):
            text_list_narr.append(element.text)
        return ('\n'.join(text_list_narr))
    except:
        return None

In [212]:
details3(teste20)

'Narrative:\nThe aircraft experienced a runway excursion upon landing at Lake County Airport (LXV/KLXV), Leadville, Colorado.  The airplane sustained substantial damage and the sole pilot onboard was not injured during the incident.'

In [107]:
def details_class(html):
    try:
        soup = BeautifulSoup(html)
        text_list2 = []
        for element in soup.find_all('div', attrs={'class':'innertube'}):
            text_list2.append(element.text)
        if len(re.findall('Classification((.|\n)*?)(Source|Photos|Follow-up|Weather|METAR)',text_list2[0]))>0:
            return re.findall('Classification((.|\n)*?)(Source|Photos|Follow-up|Weather|METAR)',text_list2[0])[0][0]
        else:
            return None
    except:
        return None

In [214]:
details4(teste20)

'Runway excursion'

In [4]:
acc_complete = pd.read_csv('accidents-2011-2021-html.csv')

In [36]:
acc_complete['narrative'] = acc_complete['html'].apply(details3)

In [108]:
acc_complete['classification'] = acc_complete['html'].apply(details_class)

In [82]:
acc_complete['inv_agency'] = acc_complete['html'].apply(details_inv)

In [45]:
acc_complete['time'] = acc_complete['html'].apply(details, index=2)

In [47]:
acc_complete['aircraft_damage'] = acc_complete['html'].apply(details, index=12)

In [48]:
acc_complete['phase'] = acc_complete['html'].apply(details, index=14)

In [49]:
acc_complete['nature'] = acc_complete['html'].apply(details, index=15)

In [109]:
acc_complete['classification'].value_counts()

:\nRunway excursion\n\n                                                     200
:\nDamaged on the ground\n\n                                                153
:\nLoss of control\n\n                                                       55
:\nRunway mishap\n\n                                                         51
:\nGround collision\nDamaged on the ground\n\n                               47
                                                                           ... 
:\n \nRunway mishap\n\n                                                       1
:\nPitot head issues\nLoss of control\n\n                                     1
:\nShot down by aircraft\n\n\n                                                1
:\nLanding gear collapse\nNormal landing\n\n                                  1
:\nLanding after unstabilized approach\nHeavy landing\nRunway mishap\n\n      1
Name: classification, Length: 400, dtype: int64

In [114]:
acc_complete['classification'].isnull()
acc_complete[acc_complete['classification'].isnull()]['url'][13]
#teste_4 = acc_complete[acc_complete['classification'].isnull()]['html'][2]

'https://aviation-safety.net/database/record.php?id=20110119-0'

In [30]:
acc_complete = acc_complete.drop(columns=['Unnamed: 0', 'Unnamed: 6','pic', 'cat'])
acc_complete

,date,type,registration,operator,fat.,location,url,html
0,01-JAN-2011,Tupolev Tu-154B-2,RA-85588,Kolavia,3,Surgut Airpo...,https://aviation-safety.net/database/record.ph...,<!-- cachefile cache/2011/cached-20110101-0.ht...
1,03-JAN-2011,Beech B200 King Air,C-GSAU,Saskatchewan Gvt.,0,Maple Creek ...,https://aviation-safety.net/database/record.ph...,<!-- cachefile cache/2011/cached-20110103-0.ht...
2,03-JAN-2011,Boeing 737-823 (WL),N831NN,American Airlines,0,Los Angeles ...,https://aviation-safety.net/database/record.ph...,<!-- cachefile cache/2011/cached-20110103-1.ht...
3,05-JAN-2011,Boeing 737-8F2 (WL),TC-JGZ,THY,0,Istanbul Ata...,https://aviation-safety.net/database/record.ph...,<!-- cachefile cache/2011/cached-20110105-0.ht...
4,06-JAN-2011,Learjet 35A,N800GP,Priester Aviation,0,Springfield-...,https://aviation-safety.net/database/record.ph...,<!-- cachefile cache/2011/cached-20110106-0.ht...
...,...,...,...,...,...,...,...,...
2187,13-NOV-2021,BN-2A-6 Islander,N866JA,Island Airways,4,near Beaver Islan...,https://aviation-safety.net/database/record.ph...,<!-- cachefile cache/2021/cached-20211113-0.ht...
2188,14-NOV-2021,IAI 1125 Astra SP,N771DX,EMD Astra Holdings LLC,0,Las Vegas-No...,https://aviation-safety.net/database/record.ph...,<!-- cachefile cache/2021/cached-20211114-0.ht...
2189,29-NOV-2021,Boeing 747-4B5F,N705CK,Kalitta Air,0,between Sant...,https://aviation-safety.net/database/record.ph...,<!-- cachefile cache/2021/cached-20211129-0.ht...
2190,01-DEC-2021,Cessna 208 Caravan I,N9602F,Lake & Peninsula Airlines,0,near Port Alswort...,https://aviation-safety.net/database/record.ph...,<!-- cachefile cache/2021/cached-20211201-0.ht...


In [51]:
acc_complete.classification.unique()

array(['Final report', 'Runway excursion', 'Hijack', 'Summary report',
       'Electrical system problem', 'Loss of control',
       'Controlled Flight Into Terrain (CFIT) - Mountain',
       'Hangar / ground fire', 'Tweet', '', 'Rejected takeoff',
       'Scramble', 'Cork Airport (ORK/EICK)', 'Soviet Transports',
       'Se estrella avioneta en Hidalgo; dos muertos', 'Gear-up landing',
       'Oslo-Gardermoen Airport (OSL/ENGM)',
       'http://www.jeremietronet.com/unions-island-kiting-fishing-surfing-and-plane-crash/',
       'Tempo.co', 'Damaged on the ground',
       "Kinshasa-N'Djili Airport (FIH/FZAA)", 'Forced landing on runway',
       'Forced landing outside airport', 'region15.ru',
       'Petrozavodsk Airport (PES/ULPB)', 'Trend', 'Undershoot/overshoot',
       'Runway mishap', 'Windshear/downdraft',
       'Raleigh/Durham International Airport, NC (RDU/KRDU)',
       'Shannon Airport (SNN/EINN)', 'Landing gear collapse',
       'Keperveyem Airport (UHMK)', 'Stabroek News',

In [53]:
html_teste = acc_complete['html'][0]

In [100]:
soup = BeautifulSoup(teste_4)
text_list2 =[]
for element in soup.find_all('div', attrs={'class':'innertube'}):
     text_list2.append(element.text)
text_list2[0]
if len(re.findall('Classification((.|\n)*?)(Source|Photos)',text_list2[0]))>0:
    print(re.findall('Classification((.|\n)*?)(Source|Photos)',text_list2[0]))
else:
    None

[(':\nTailstrike\nRunway mishap\n\n\n\n \n', '\n', 'Photos')]


In [99]:
text_list2

['\n\n\n\n\nStatus:Final\nDate:Monday 3 January 2011\nTime:10:17\nType:Boeing 737-823 (WL)\nOperator:American Airlines\nRegistration: N831NN\nMSN: 33211/3217\nFirst flight: 2010-03-09  (10 months)\nEngines: 2 CFMI CFM56-7B24/3\nCrew:Fatalities: 0 / Occupants: 5\nPassengers:Fatalities: 0 / Occupants: 160\nTotal:Fatalities: 0 / Occupants: 165 \nAircraft damage: Substantial\nAircraft fate: Repaired\nLocation:Los Angeles International Airport, CA (LAX) ( \xa0 United States of America) \r\n\nPhase: Takeoff (TOF)\nNature:International Scheduled Passenger\nDeparture airport:Los Angeles International Airport, CA (LAX/KLAX), United States of America\nDestination airport:Toronto-Pearson International Airport, ON (YYZ/CYYZ), Canada\nFlightnumber:AA1586\n\nNarrative:American Airlines flight 1586, a Boeing 737-800, experienced a tailstrike upon takeoff from runway 7L at Los Angeles International Airport, California, USA. No injuries occurred during the event. The flight landed uneventfully at Toron

In [56]:
len(text_list2)

2